<a href="https://colab.research.google.com/github/XavierRG0417/ETL/blob/main/etl_bcb_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ETL Automatizado para Análise de Dados Bancários: Simplificando o Tratamento e Integração de Dados Financeiros com Python**

---

## **Introdução**

Este projeto apresenta um pipeline de ETL (Extração, Transformação e Carga) desenvolvido em Python, projetado para automatizar o processamento e a integração de dados financeiros do setor bancário brasileiro. Utilizando a API do Banco Central como fonte de dados, o script organiza informações financeiras em tabelas padronizadas e prontas para análises avançadas, otimizando atividades como planejamento financeiro, relatórios regulatórios e modelagem econométrica.  

---

### **Objetivo Principal**
Facilitar o trabalho com dados extensos e diversos, utilizando Python para:
- **Automatizar** a extração, transformação e organização de dados.
- **Reduzir erros** associados ao tratamento manual de informações.
- **Garantir escalabilidade**, permitindo a reutilização do pipeline para diferentes tipos de relatórios.
- **Facilitar análises quantitativas**, com dados estruturados e consistentes.

---

## **Características do Projeto**
- **Extração de Dados**:
    - Coleta informações diretamente da API do Banco Central, como Ativo, Passivo, Demonstrações de Resultados e Resumo.
- **Transformação e Limpeza**:
    - Estrutura tabelas pivotadas e padronizadas para análise.
- **Armazenamento**:
    - Exportação de dados organizados em arquivos Excel.
- **Modularidade e Reutilização**:
    - Funções adaptáveis para expansão a novos relatórios ou fontes de dados.

---

## **Benefícios do ETL Automatizado**
1. **Eficiência**: Processa grandes volumes de dados rapidamente, eliminando a necessidade de manipulações manuais.
2. **Consistência**: Garante a padronização e a confiabilidade dos dados financeiros.
3. **Flexibilidade**: Permite reaproveitamento em diferentes contextos financeiros e analíticos.
4. **Redução de Erros**: Automatiza etapas críticas, minimizando falhas humanas.

---

## **Estrutura do Script**
O pipeline está estruturado em três etapas principais:
1. **Configuração Inicial**:
    - Definição de bibliotecas e variáveis iniciais.
2. **Funções Reutilizáveis**:
    - **Coleta de dados**: Scripts que acessam e organizam informações da API.
    - **Transformação**: Criação de tabelas pivotadas e padronizadas.
    - **Armazenamento**: Exportação de relatórios para arquivos Excel.
3. **Execução do Pipeline**:
    - Integração das funções para gerar relatórios financeiros completos.

---

## **Como Usar Este Projeto**
1. **Configuração Prévia**:
    - Verifique as credenciais e URLs das APIs utilizadas.
2. **Execução**:
    - Execute o script para processar e exportar os relatórios financeiros.
3. **Análise**:
    - Utilize os arquivos Excel gerados para análise financeira, planejamento estratégico ou relatórios regulatórios.

---

### **Próximos Passos**
O pipeline pode ser expandido para incluir:
- Integração com dashboards interativos para visualização dinâmica.
- Modelagem avançada, como projeções financeiras e indicadores de desempenho (KPIs).
- Automação de novos fluxos de dados regulatórios.

---

### **Autor**
Projeto desenvolvido por Gabriel Xavier, automatizando o tratamento de dados financeiros para análises robustas e práticas no setor bancário brasileiro.

In [ ]:
# Imporatando as bibliotecas
import requests
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from linearmodels import PanelOLS
from linearmodels.panel import PooledOLS
import statsmodels.api as sm
import datetime as dt

In [ ]:
# Criando uma lista com os trimestres

quarters = [
    '201203', '201206', '201209', '201212',
    '201303', '201306', '201309', '201312',
    '201403', '201406', '201409', '201412',
    '201503', '201506', '201509', '201512',
    '201603', '201606', '201609', '201612',
    '201703', '201706', '201709', '201712',
    '201803', '201806', '201809', '201812',
    '201903', '201906', '201909', '201912',
    '202003', '202006', '202009', '202012',
    '202103', '202106', '202109', '202112',
    '202203', '202206', '202209','202212',
    '202303', '202306', '202309','202312',
    '202403', '202406'

]

In [ ]:
# Função para Coletar Dados de API
def fetch_data(url):
    """
    Faz uma requisição à API e retorna um DataFrame.
    """
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return pd.DataFrame(data['value'])
    else:
        print(f"Erro ao acessar {url}")
        return pd.DataFrame()

# Função para Processar Dados por Relatório (Ativo, Passivo, DRE, Resumo)
def process_data(quarter, report_type, relatorio_nome):
    """
    Extrai e organiza os dados de um trimestre para o tipo de relatório.
    """
    # URL para Cadastro de Instituições
    institution_url = (
        f"https://olinda.bcb.gov.br/olinda/servico/IFDATA/versao/v1/odata/"
        f"IfDataCadastro(AnoMes=@AnoMes)?@AnoMes={quarter}&$top=10000&$format=json"
    )
    institutions = fetch_data(institution_url)
    if institutions.empty:
        return pd.DataFrame()

    institutions = institutions[['CodInst', 'NomeInstituicao', 'Atividade', 'Municipio', 'Situacao']]
    institutions = institutions.drop_duplicates(subset=['CodInst', 'NomeInstituicao'])
    institutions = institutions.rename(columns={'CodInst': 'ID', 'NomeInstituicao': 'Inst Financeira'})

    # URL para Dados de Relatório
    data_url = (
        f"https://olinda.bcb.gov.br/olinda/servico/IFDATA/versao/v1/odata/"
        f"IfDataValores(AnoMes=@AnoMes,TipoInstituicao=@TipoInstituicao,Relatorio=@Relatorio)"
        f"?@AnoMes={quarter}&@TipoInstituicao=2&@Relatorio='{report_type}'&$top=99999&$format=json"
    )
    data = fetch_data(data_url)
    if data.empty:
        return pd.DataFrame()

    # Limpeza e Transformação
    data = data[['CodInst', 'AnoMes', 'NomeColuna', 'Saldo']]
    data = data.rename(columns={'CodInst': 'ID', 'AnoMes': 'Data'})
    data = data.merge(institutions, on='ID', how='left')
    data['Data'] = pd.to_datetime(data['Data'], format='%Y%m')

    # Pivot Table
    pivot_table = pd.pivot_table(data, index=['ID', 'Inst Financeira', 'Data'], columns='NomeColuna', values='Saldo')
    return pivot_table

# Função de ETL para Relatórios
def etl_report(report_type, file_name, relatorio_nome):
    """
    Executa o pipeline de ETL para um tipo de relatório e salva o resultado.
    """
    combined_data = []
    for quarter in quarters:
        print(f"Processando {relatorio_nome} para o trimestre {quarter}...")
        data = process_data(quarter, report_type, relatorio_nome)
        if not data.empty:
            combined_data.append(data)

    final_data = pd.concat(combined_data).reset_index()
    final_data.to_excel(file_name, index=False)
    print(f"Arquivo {file_name} gerado com sucesso!")
    return final_data

# Função Principal para Executar Todo o Pipeline
def main():
    # Processar Ativo
    ativo_data = etl_report(report_type='2', file_name="Ativo_nominal.xlsx", relatorio_nome="Ativo")

    # Processar Passivo
    passivo_data = etl_report(report_type='3', file_name="Passivo_nominal.xlsx", relatorio_nome="Passivo")

    # Processar Resumo
    resumo_data = etl_report(report_type='1', file_name="Resumo.xlsx", relatorio_nome="Resumo")

    # Processar DRE
    dre_data = etl_report(report_type='4', file_name="DRE_nominal.xlsx", relatorio_nome="DRE")

# Executar o Script
if __name__ == "__main__":
    main()

Processando Ativo para o trimestre 201203...
Processando Ativo para o trimestre 201206...
Processando Ativo para o trimestre 201209...
Processando Ativo para o trimestre 201212...
Processando Ativo para o trimestre 201303...
Processando Ativo para o trimestre 201306...
Processando Ativo para o trimestre 201309...
Processando Ativo para o trimestre 201312...
Processando Ativo para o trimestre 201403...
Processando Ativo para o trimestre 201406...
Processando Ativo para o trimestre 201409...
Processando Ativo para o trimestre 201412...
Processando Ativo para o trimestre 201503...
Processando Ativo para o trimestre 201506...
Processando Ativo para o trimestre 201509...
Processando Ativo para o trimestre 201512...
Processando Ativo para o trimestre 201603...
Processando Ativo para o trimestre 201606...
Processando Ativo para o trimestre 201609...
Processando Ativo para o trimestre 201612...
Processando Ativo para o trimestre 201703...
Processando Ativo para o trimestre 201706...
Processand